## Format variant data to fetch variant effects and consequences via PredictSNP2 software

The variant data should be in the following structure:

CHR,START,[END,]REFERENCE_ALLELE,ALTERNATIVE_ALLELE



In [1]:
# Import modules and packages

import os
import pandas as pd
import sys
sys.path.append(r"C:\Users\User\Desktop\Megan\MSC2\Results\5._Posthoc_analysis\Pipeline_GnomAD_SAHGP_14032023\Genomic_data_analysis")
import Utils.constants as constants
import Utils.functions as functions

In [13]:
# Import variant data. Variant data is split across two different types of tables. Import both into pandas dataframes and merge.
collated_variant_df = pd.DataFrame()
for gene in constants.GENES:
    variant_info_path = os.path.join(constants.HOME_PATH,"Data", "SUPER", "ALL_{}.acount".format(gene))
    variant_info_path2 = os.path.join(constants.HOME_PATH,"Data", "Raw", "SUPER", "{}_Count.csv".format(gene))
    if os.path.exists(variant_info_path):
        variant_info_df = pd.read_csv(variant_info_path, sep="\t")[["#CHROM","ID", "REF", "ALT"]]
        variant_info_df2 = pd.read_csv(variant_info_path2, sep="\t")[["ID", "POS", "REF", "ALT"]]
        variant_info_df["GENE"] = gene
        variant_info_df2["GENE"] = gene
        variant_info = pd.merge(left=variant_info_df, right=variant_info_df2, on=["ID","REF","ALT","GENE"])
    collated_variant_df = pd.concat([collated_variant_df, variant_info]).reset_index(drop=True)

collated_variant_df.head(5)

,#CHROM,ID,REF,ALT,GENE,POS
0,13,chr13:110148882C-CT,C,CT,COL4A1,110148882
1,13,rs552586867,C,G,COL4A1,110148891
2,13,rs59409892,C,G,COL4A1,110148917
3,13,rs535182970,G,C,COL4A1,110148920
4,13,rs56406633,A,G,COL4A1,110148959


In [12]:
# Format variants into input for PredictSNP2 software

collated_variant_df = collated_variant_df.rename(columns={"#CHROM":"CHROM"})
for index, row in collated_variant_df.iterrows():
    if len(row["REF"]) != 1 or len(row["ALT"]) != 1:
        collated_variant_df.drop(index, inplace=True)
        
predictsnp2_input = open("predictsnp2_input.txt", "w")
for variant in collated_variant_df.ID:
    variant_entry = collated_variant_df[collated_variant_df.ID == variant]
    predictsnp2_format = "chr" + variant_entry.CHROM.values[0].astype(str) + "," + variant_entry.POS.values[0].astype(str) + "," + variant_entry.REF.values[0] + "," + variant_entry.ALT.values[0]
    predictsnp2_input.write(predictsnp2_format + "\n")